In [427]:
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone as PL
from loguru import logger as logging

from dotenv import load_dotenv
load_dotenv()

True

In [428]:
model_name = "BAAI/bge-m3"

embedding_function = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)

In [25]:
news_df = pd.read_csv('../lenta-ru-news.csv')
loader = DataFrameLoader(news_df, page_content_column='text')
documents = loader.load()
logging.info(f'documents: {len(documents)}')

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 420, chunk_overlap = 380)
docs = text_splitter.split_documents(documents)
logging.info(f'docs: {len(docs)}')


C:\Users\ekurochkina\AppData\Local\Temp2\ipykernel_19968\3880527182.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  news_df = pd.read_csv('../lenta-ru-news.csv')
2024-09-12 07:58:12.137 | INFO     | __main__:<module>:4 - documents: 800975
2024-09-12 08:03:44.406 | INFO     | __main__:<module>:8 - docs: 19296165


In [30]:
index_name = 'index-full-420'

logging.info(f'from: {index_name}')
docsearch = PL.from_existing_index(index_name, embedding_function)

2024-09-12 08:05:08.746 | INFO     | __main__:<module>:4 - from: index-full-420


In [7]:
retriever = docsearch.as_retriever(search_type="similarity",
                                   search_kwargs={"k": 5})

In [429]:
data_samples = {
    'question':
        ['Чем закончились бои у Сопоцкина и Друскеник?',
         "Как помогал бульдог лейтенанту бельгийской артиллерии?",
         "Источник заражения холеры на Южно-Сахалинске?",
         "Сколько человек участвовали в боях в Дагестане?",
         "Новость про российских хакеров, которые взламывают Пентагон",
         "*Новость про российских хакеров, которые взламывают Пентагон"],  #for test with wrong ground-truth

    'ground_truth':
        ['Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к',

         'животных. Лейтенант бельгийской артиллерии, руководивший обороной одного из фортов Льежа, ни за что не хотел расстаться с своей собакой — бульдогом. Когда пруссаки пробрались между фортов в самый город, офицеру пришло в голову доверить бульдогу письмо, в котором он посылал успокоительную весть своим родителям. Благородный пес честно исполнил свою миссию. Десять часов спустя бульдог проник обратно в форт и принес',

         'холеры. Как сообщает ИТАР-ТАСС со ссылкой на пресс-центр администрации Сахалинской области, в лечебных учреждениях Южно-Сахалинска уже находятсятся 5 горожан, причем у двоих из них болезнь проходит в средне-тяжелой форме. Специалисты Госсанэпиднадзора по Сахалинской области и Хабаровской противочумной станции считают, что возможным источником заражения стала река Хомутовка в военном городке и слабосоленная красная',

         'В МВД России, сообщает ИТАР-ТАСС, имеется подтвержденная информация о том, что в боевых действиях в Дагестане принимали активное участие официальные силовые структуры Чеченской республики. Как сообщили в пресс-центре МВД, в боях в Дагестане участвовали около 300 человек из войсковой части №0666 ВС ЧРИ в Гудермесе, которая непосредственно подчиняется президенту Аслану Масхадову. Согласно оперативным данным, 11',

         'описал ситуацию следующим образом: "Мы находимся в состоянии кибер-войны". Как полагает Пентагон, массированное наступление на компьютеры министерства обороны и министерства энергетики США ведут хакеры из Российской Академии наук, которая связана с военными лабораториями и спонсируется правительством. Как отмечает Newsweek, на сегодняшний день это первый случай сообщения о хакерской войне, которая ведется именно по',

         'Как полагает Пентагон, массированное наступление на компьютеры министерства обороны и министерства энергетики США ведут хакеры из Российской Академии наук, которая связана с военными лабораториями и спонсируется правительством. Как отмечает Newsweek, на сегодняшний день это первый случай сообщения о хакерской войне, которая ведется именно по государственному заказу, а не отдельными независимыми хакерами.  По словам']
}

eval_df = pd.DataFrame(data_samples)
eval_df

,question,ground_truth
0,Чем закончились бои у Сопоцкина и Друскеник?,Бои у Сопоцкина и Друскеник закончились отступ...
1,Как помогал бульдог лейтенанту бельгийской арт...,"животных. Лейтенант бельгийской артиллерии, ру..."
2,Источник заражения холеры на Южно-Сахалинске?,холеры. Как сообщает ИТАР-ТАСС со ссылкой на п...
3,Сколько человек участвовали в боях в Дагестане?,"В МВД России, сообщает ИТАР-ТАСС, имеется подт..."
4,"Новость про российских хакеров, которые взламы...","описал ситуацию следующим образом: ""Мы находим..."
5,"*Новость про российских хакеров, которые взлам...","Как полагает Пентагон, массированное наступлен..."


In [430]:
def perform_retrieval(df):
    for index, row in tqdm(df.iterrows()):
        question = row['question']
        df.at[index, 'retriever_answer'] = ''
        retriever = docsearch.as_retriever(search_type="similarity",
                                           search_kwargs={"k": 5})
        answer = retriever.invoke(question)
        answer_list = [a.page_content for a in answer]
        df.at[index, 'retriever_answer'] = answer_list
    return df

eval_df = perform_retrieval(df=eval_df)
eval_df

6it [00:02,  2.25it/s]


,question,ground_truth,retriever_answer
0,Чем закончились бои у Сопоцкина и Друскеник?,Бои у Сопоцкина и Друскеник закончились отступ...,[Бои у Сопоцкина и Друскеник закончились отсту...
1,Как помогал бульдог лейтенанту бельгийской арт...,"животных. Лейтенант бельгийской артиллерии, ру...","[животных. Лейтенант бельгийской артиллерии, р..."
2,Источник заражения холеры на Южно-Сахалинске?,холеры. Как сообщает ИТАР-ТАСС со ссылкой на п...,[холеры. Как сообщает ИТАР-ТАСС со ссылкой на ...
3,Сколько человек участвовали в боях в Дагестане?,"В МВД России, сообщает ИТАР-ТАСС, имеется подт...","[В МВД России, сообщает ИТАР-ТАСС, имеется под..."
4,"Новость про российских хакеров, которые взламы...","описал ситуацию следующим образом: ""Мы находим...","[описал ситуацию следующим образом: ""Мы находи..."
5,"*Новость про российских хакеров, которые взлам...","Как полагает Пентагон, массированное наступлен...",[- срочно сменить пароли на своих машинах. Зам...


## Metrics for Evaluation

**1. Precision@K**
--- measures the proportion of relevant instances among the retrieved instances.

**2. Recall@K**
--- the proportion of relevant items that were successfully recommended or retrieved within the top k items.

In [431]:
def precision_at_k(y_true, y_pred, k):
    pred = y_pred[:k]
    relevant = sum([1 for y in pred if y in y_true])
    return round(relevant / len(pred), 2)

def recall_at_k(y_true, y_pred, k):
    true_set = set([y_true])
    pred_set = set(y_pred[:k])
    if len(true_set & pred_set) == 0:
        return 0
    result = round(len(true_set & pred_set) / float(len(true_set)), 2)
    return result

def calculate_recall(k_list, data):
    df = data.copy()
    for i in range(len(k_list)):
        k = k_list[i]
        for index, row in df.iterrows():
            act = row['ground_truth']
            pred = row['retriever_answer']
            column_recall_name = 'recall_k' + str(k)
            column_prec_name = 'precision_k' + str(k)
            df.at[index, column_prec_name] =  precision_at_k(act, pred, k=k)
            df.at[index, column_recall_name] =  recall_at_k(act, pred, k=k)
    return df


top_k = [1, 2, 3, 5]
scores_df = calculate_recall(k_list=top_k, data=eval_df)
scores_df

,question,ground_truth,retriever_answer,precision_k1,recall_k1,precision_k2,recall_k2,precision_k3,recall_k3,precision_k5,recall_k5
0,Чем закончились бои у Сопоцкина и Друскеник?,Бои у Сопоцкина и Друскеник закончились отступ...,[Бои у Сопоцкина и Друскеник закончились отсту...,1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0
1,Как помогал бульдог лейтенанту бельгийской арт...,"животных. Лейтенант бельгийской артиллерии, ру...","[животных. Лейтенант бельгийской артиллерии, р...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0
2,Источник заражения холеры на Южно-Сахалинске?,холеры. Как сообщает ИТАР-ТАСС со ссылкой на п...,[холеры. Как сообщает ИТАР-ТАСС со ссылкой на ...,1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0
3,Сколько человек участвовали в боях в Дагестане?,"В МВД России, сообщает ИТАР-ТАСС, имеется подт...","[В МВД России, сообщает ИТАР-ТАСС, имеется под...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0
4,"Новость про российских хакеров, которые взламы...","описал ситуацию следующим образом: ""Мы находим...","[описал ситуацию следующим образом: ""Мы находи...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0
5,"*Новость про российских хакеров, которые взлам...","Как полагает Пентагон, массированное наступлен...",[- срочно сменить пароли на своих машинах. Зам...,0.0,0.0,0.0,0.0,0.33,1.0,0.2,1.0


In [450]:
eval_df.retriever_answer[0]

['Бои у Сопоцкина и Друскеник закончились отступлением германцев. Неприятель, приблизившись с севера к Осовцу начал артиллерийскую борьбу с крепостью. В артиллерийском бою принимают участие тяжелые калибры. С раннего утра 14 сентября огонь достиг значительного напряжения. Попытка германской пехоты пробиться ближе к крепости отражена. В Галиции мы заняли Дембицу. Большая колонна, отступавшая по шоссе от Перемышля к',
 'Девятинского переулка проник на территорию посольства.Увидев там машину c оставленными ключами, он попытался ее завести и выехать. Внутренняя охрана - американские морские пехотинцы - произвела несколько предупредительных выстрелов в воздух, но поскольку Тайнаков никак не отреагировал, открыла огонь на поражение. Тяжелораненый Тайнаков был отправлен представителями властей в клинику Склифосовского, где и был',
 '- американские морские пехотинцы - произвела несколько предупредительных выстрелов в воздух, но поскольку Тайнаков никак не отреагировал, открыла огонь на поражен

**3. Mean Reciprocal Rank (MRR)**
---  takes the order of relevant items into consideration. It is particularly focused on the position of the first relevant item in the ranked list of results, providing a more fine-grained evaluation in the context of information retrieval systems.

In [432]:
def mean_reciprocal_rank(y_true, y_pred):
    def reciprocal_rank(y_true, y_pred):
        for i, p in enumerate(y_pred):
            if p in y_true:
                return 1 / (i + 1)
        return 0

    rr = [reciprocal_rank(yt, yp) for yt, yp in zip(y_true, y_pred)]
    return round(sum(rr) / len(rr), 3)

def add_MRR(df):
    for index, row in df.iterrows():
        act = [row['ground_truth']]
        pred = [row['retriever_answer']]
        col_name = 'MRR'
        mrr_val = mean_reciprocal_rank(act, pred)
        df.at[index, col_name] =  mrr_val
        print(f'question {index+1}: MRR = {mrr_val}')
    return df

scores_df = add_MRR(scores_df)
scores_df

question 1: MRR = 1.0
question 2: MRR = 1.0
question 3: MRR = 1.0
question 4: MRR = 1.0
question 5: MRR = 1.0
question 6: MRR = 0.333


,question,ground_truth,retriever_answer,precision_k1,recall_k1,precision_k2,recall_k2,precision_k3,recall_k3,precision_k5,recall_k5,MRR
0,Чем закончились бои у Сопоцкина и Друскеник?,Бои у Сопоцкина и Друскеник закончились отступ...,[Бои у Сопоцкина и Друскеник закончились отсту...,1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000
1,Как помогал бульдог лейтенанту бельгийской арт...,"животных. Лейтенант бельгийской артиллерии, ру...","[животных. Лейтенант бельгийской артиллерии, р...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000
2,Источник заражения холеры на Южно-Сахалинске?,холеры. Как сообщает ИТАР-ТАСС со ссылкой на п...,[холеры. Как сообщает ИТАР-ТАСС со ссылкой на ...,1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000
3,Сколько человек участвовали в боях в Дагестане?,"В МВД России, сообщает ИТАР-ТАСС, имеется подт...","[В МВД России, сообщает ИТАР-ТАСС, имеется под...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000
4,"Новость про российских хакеров, которые взламы...","описал ситуацию следующим образом: ""Мы находим...","[описал ситуацию следующим образом: ""Мы находи...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000
5,"*Новость про российских хакеров, которые взлам...","Как полагает Пентагон, массированное наступлен...",[- срочно сменить пароли на своих машинах. Зам...,0.0,0.0,0.0,0.0,0.33,1.0,0.2,1.0,0.333


In [433]:
# Example usage with the wrong pairs
true = [[scores_df.ground_truth[1]]]
pred = [scores_df.retriever_answer[2]]
print(f'test MRR: {mean_reciprocal_rank(true, pred)}')

true = [[scores_df.ground_truth[4]]]
pred = [scores_df.retriever_answer[5]]
print(f'test MRR: {mean_reciprocal_rank(true, pred)}')

test MRR: 0.0
test MRR: 0.5


**4. Mean Average Precision (MAP)**
--- is the mean of the Average Precision (AP) scores for a set of queries. AP is the average of precision values at the ranks where relevant documents occur.

In [434]:
def average_precision(y_true, y_pred):
    relevant = 0
    precisions = []
    for i, p in enumerate(y_pred):
        if p in y_true:
            relevant += 1
            precisions.append(relevant / (i + 1))
    if not precisions:
        return 0
    return sum(precisions) / len(y_true)

def mean_average_precision(y_true, y_pred):
    ap = [average_precision(yt, yp) for yt, yp in zip(y_true, y_pred)]
    return round(sum(ap) / len(ap), 3)


def add_MAP(df):
    for index, row in df.iterrows():
        act = [[row['ground_truth']]]
        pred = [row['retriever_answer']]
        col_name = 'MAP'
        map_val = mean_average_precision(act, pred)
        df.at[index, col_name] =  map_val
        print(f'{index+1}: MAP = {map_val}')
    return df

scores_df = add_MAP(scores_df)
scores_df

1: MAP = 1.0
2: MAP = 1.0
3: MAP = 1.0
4: MAP = 1.0
5: MAP = 1.0
6: MAP = 0.333


,question,ground_truth,retriever_answer,precision_k1,recall_k1,precision_k2,recall_k2,precision_k3,recall_k3,precision_k5,recall_k5,MRR,MAP
0,Чем закончились бои у Сопоцкина и Друскеник?,Бои у Сопоцкина и Друскеник закончились отступ...,[Бои у Сопоцкина и Друскеник закончились отсту...,1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000,1.000
1,Как помогал бульдог лейтенанту бельгийской арт...,"животных. Лейтенант бельгийской артиллерии, ру...","[животных. Лейтенант бельгийской артиллерии, р...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000,1.000
2,Источник заражения холеры на Южно-Сахалинске?,холеры. Как сообщает ИТАР-ТАСС со ссылкой на п...,[холеры. Как сообщает ИТАР-ТАСС со ссылкой на ...,1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000,1.000
3,Сколько человек участвовали в боях в Дагестане?,"В МВД России, сообщает ИТАР-ТАСС, имеется подт...","[В МВД России, сообщает ИТАР-ТАСС, имеется под...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000,1.000
4,"Новость про российских хакеров, которые взламы...","описал ситуацию следующим образом: ""Мы находим...","[описал ситуацию следующим образом: ""Мы находи...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000,1.000
5,"*Новость про российских хакеров, которые взлам...","Как полагает Пентагон, массированное наступлен...",[- срочно сменить пароли на своих машинах. Зам...,0.0,0.0,0.0,0.0,0.33,1.0,0.2,1.0,0.333,0.333


In [435]:
# Example usage with the wrong pairs
true = [[scores_df.ground_truth[1]]]
pred = [scores_df.retriever_answer[2]]
print(f'test MAP: {mean_average_precision(true, pred)}')

true = [[scores_df.ground_truth[4]]]
pred = [scores_df.retriever_answer[5]]
print(f'test MAP: {mean_average_precision(true, pred)}')

test MAP: 0.0
test MAP: 0.5


**5. Normalized Discounted Cumulative Gain (NDCG)**
--- measures the gain of a document based on its position in the result list, normalized by the ideal DCG.

In [436]:
def dcg_at_k(y_true, y_pred, k):
    y_pred = y_pred[:k]
    gain = 0
    for i, p in enumerate(y_pred):
        if p in y_true:
            gain += 1 / np.log2(i + 2)
    return gain

def ndcg_at_k(y_true, y_pred, k):
    ideal_dcg = dcg_at_k(y_true, y_true, k)
    actual_dcg = dcg_at_k(y_true, y_pred, k)
    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0

def add_ndcg(k_list, df):
    for i in range(len(k_list)):
        k = k_list[i]
        for index, row in df.iterrows():
            act = [row['ground_truth']]
            pred = row['retriever_answer']
            col_name = 'NDSG@' + str(k)
            ndcg_val = ndcg_at_k(act, pred, k)
            df.at[index, col_name] =  ndcg_val
            print(f'question {index+1}: {col_name} = {ndcg_val}')
    return df

top_k = [1, 2, 3, 5]
scores_df = add_ndcg(top_k, scores_df)
scores_df

question 1: NDSG@1 = 1.0
question 2: NDSG@1 = 1.0
question 3: NDSG@1 = 1.0
question 4: NDSG@1 = 1.0
question 5: NDSG@1 = 1.0
question 6: NDSG@1 = 0.0
question 1: NDSG@2 = 1.0
question 2: NDSG@2 = 1.0
question 3: NDSG@2 = 1.0
question 4: NDSG@2 = 1.0
question 5: NDSG@2 = 1.0
question 6: NDSG@2 = 0.0
question 1: NDSG@3 = 1.0
question 2: NDSG@3 = 1.0
question 3: NDSG@3 = 1.0
question 4: NDSG@3 = 1.0
question 5: NDSG@3 = 1.0
question 6: NDSG@3 = 0.5
question 1: NDSG@5 = 1.0
question 2: NDSG@5 = 1.0
question 3: NDSG@5 = 1.0
question 4: NDSG@5 = 1.0
question 5: NDSG@5 = 1.0
question 6: NDSG@5 = 0.5


,question,ground_truth,retriever_answer,precision_k1,recall_k1,precision_k2,recall_k2,precision_k3,recall_k3,precision_k5,recall_k5,MRR,MAP,NDSG@1,NDSG@2,NDSG@3,NDSG@5
0,Чем закончились бои у Сопоцкина и Друскеник?,Бои у Сопоцкина и Друскеник закончились отступ...,[Бои у Сопоцкина и Друскеник закончились отсту...,1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000,1.000,1.0,1.0,1.0,1.0
1,Как помогал бульдог лейтенанту бельгийской арт...,"животных. Лейтенант бельгийской артиллерии, ру...","[животных. Лейтенант бельгийской артиллерии, р...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000,1.000,1.0,1.0,1.0,1.0
2,Источник заражения холеры на Южно-Сахалинске?,холеры. Как сообщает ИТАР-ТАСС со ссылкой на п...,[холеры. Как сообщает ИТАР-ТАСС со ссылкой на ...,1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000,1.000,1.0,1.0,1.0,1.0
3,Сколько человек участвовали в боях в Дагестане?,"В МВД России, сообщает ИТАР-ТАСС, имеется подт...","[В МВД России, сообщает ИТАР-ТАСС, имеется под...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000,1.000,1.0,1.0,1.0,1.0
4,"Новость про российских хакеров, которые взламы...","описал ситуацию следующим образом: ""Мы находим...","[описал ситуацию следующим образом: ""Мы находи...",1.0,1.0,0.5,1.0,0.33,1.0,0.2,1.0,1.000,1.000,1.0,1.0,1.0,1.0
5,"*Новость про российских хакеров, которые взлам...","Как полагает Пентагон, массированное наступлен...",[- срочно сменить пароли на своих машинах. Зам...,0.0,0.0,0.0,0.0,0.33,1.0,0.2,1.0,0.333,0.333,0.0,0.0,0.5,0.5


In [437]:
# Example usage with several ground-truth answers
y_true =  [scores_df.ground_truth[4]] + [scores_df.ground_truth[0]] + [scores_df.ground_truth[2]]
y_pred = scores_df.retriever_answer[2]

for n in top_k:
    print(f'test NDCG@{n}: {ndcg_at_k(y_true, y_pred, k=n)}')

test NDCG@1: 1.0
test NDCG@2: 0.6131471927654584
test NDCG@3: 0.46927872602275644
test NDCG@5: 0.46927872602275644
